In this Notebook I do data transformation, handle missing values and preprocess the data to use it in the next step, exploratory analysis. Here I also do some EDA but only for the purpose of data wrangling. I do the main EDA on eda.ipynb notebook.

In [9]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('data/field_trap_data.csv')
df.head()

,trap_id_id,date,crop_name,region,pest,trap_count,form_submission_date,is_sticky_sheet_replaced,lat,lon
0,27,5/25/20,Broccoli,California,Diamondback moth,79.0,5/25/20,True,35.188384,-118.826319
1,27,5/26/20,Broccoli,California,Diamondback moth,NaN,NaN,False,35.188384,-118.826319
2,27,5/27/20,Broccoli,California,Diamondback moth,NaN,NaN,False,35.188384,-118.826319
3,27,5/28/20,Broccoli,California,Diamondback moth,NaN,NaN,False,35.188384,-118.826319
4,27,5/29/20,Broccoli,California,Diamondback moth,112.0,5/29/20,True,35.188384,-118.826319


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605 entries, 0 to 604
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   trap_id_id                605 non-null    int64  
 1   date                      605 non-null    object 
 2   crop_name                 605 non-null    object 
 3   region                    423 non-null    object 
 4   pest                      605 non-null    object 
 5   trap_count                160 non-null    float64
 6   form_submission_date      160 non-null    object 
 7   is_sticky_sheet_replaced  605 non-null    bool   
 8   lat                       605 non-null    float64
 9   lon                       605 non-null    float64
dtypes: bool(1), float64(3), int64(1), object(5)
memory usage: 43.3+ KB


## Transforming the data

In [12]:
# Let's convert the date fields into datetime object format
df.loc[:,'date'] = pd.to_datetime(df['date'])
df.loc[:, 'form_submission_date'] = pd.to_datetime(df['form_submission_date'])

In [17]:
# Let's convert the boolean field (is_sticky_sheet_replaced) to binary numbers. 
# This will be our dummy variable that we can use as a predictor.
df.loc[:,'is_sticky_sheet_replaced'] = df['is_sticky_sheet_replaced']\
    .map({True:1, False:0}).sum()

53

In [63]:
print(f'Number of traps {df["trap_id_id"].nunique()}')
print(f'Number of unique longtitude {df["lon"].nunique()}')
print(f'Number of unique longtitude {df["lat"].nunique()}')

Number of traps 10
Number of unique longtitude 10
Number of unique longtitude 10


Since it is very unlikely to have the same exact latitude or longtitude for two or more different fields, we can safely assume that there are only one trap on each field and therefore, trap_id_id can be used as a proxy for unique field id instead of lat, lon columns or creating another unique identifier for fields.

### Some Questions
Since there are two dates field in the data (one for the dates traps checked and the other for the date the forms submitted), we need to understand which of these fields provide a time series dimension of the data set. The answer to these could be both or only one of them. To find the relavant time components of the data, we should answer the following questions:
1. Are the sticky sheets replaced only when the form submitted?
2. Are the trap counts recorded only when the form submitted?

In [52]:
print('Filtered by the form submission date')
num_null_sticky_sheet = df[df['form_submission_date'].notna()]['trap_count'].isna().sum()
print(f"The number of missing values for trap_count = {num_null_sticky_sheet} ")
df[df['form_submission_date'].notna()]['is_sticky_sheet_replaced'].value_counts()

Filtered by the form submission date
The number of missing values for trap_count = 0 


False    107
True      53
Name: is_sticky_sheet_replaced, dtype: int64

In [53]:
print('Filtered by the dates without form submission')
num_null_sticky_sheet = df[df['form_submission_date'].isna()]['trap_count'].isna().sum()
print(f"The number of missing values for trap_count = {num_null_sticky_sheet} ")
df[df['form_submission_date'].isna()]['is_sticky_sheet_replaced'].value_counts()

Filtered by the dates without form submission
The number of missing values for trap_count = 445 


False    445
Name: is_sticky_sheet_replaced, dtype: int64

## Answers to questions above
1. Are the sticky sheets replaced only when the form submitted?
* Sticky sheets have been replaced only on the dates forms submitted but not all the times when the forms submitted (53 times out of 160 form submission)

2. Are the trap counts recorded only when the form submitted
* Yes

**Answer to these questions indicate that `date` field does not provide any useful information. Only the Non-null `form_submission_date` can be used to drive actionable insights from the data** 

Therefore for further analysis we can consider only the dates the forms submitted.

In [57]:
df = df.dropna(subset=['form_submission_date']).reset_index(drop=True)

Let's see if `date` and `form_submission_date` columns are equal.

In [68]:
mask = df['date'] == df['form_submission_date']
df[~mask]

,trap_id_id,date,crop_name,region,pest,trap_count,form_submission_date,is_sticky_sheet_replaced,lat,lon
27,84,2020-06-25,Brussels sprout,California,Diamondback moth,20.0,2020-06-26,False,36.687560,-121.791370
31,84,2020-07-21,Brussels sprout,California,Diamondback moth,71.0,2020-07-22,False,36.687560,-121.791370
39,89,2020-06-19,Broccoli,NaN,Diamondback moth,76.0,2020-06-22,False,36.191092,-121.095686
78,94,2020-07-08,Brussels sprout,California,Diamondback moth,31.0,2020-07-15,True,36.765970,-121.758380
79,94,2020-07-17,Brussels sprout,California,Diamondback moth,37.0,2020-07-20,True,36.765970,-121.758380
113,108,2020-06-26,Brussels sprout,California,Diamondback moth,24.0,2020-06-27,True,36.527295,-121.452742


Occasionally forms submitted after the field visit. However, we can still drop the `form_submission_date` and use `date` column as time index 

In [77]:
# del df['form_submission_date']
# Let's save the final version of the data for the next step
df.to_csv('data/pre_processed.csv',index=False)